In [ ]:
#Instalar e Configurar o H2O
#Importar e Inicializar o H2O

import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ODeepLearningEstimator

# Iniciar o H2O
h2o.init()


In [ ]:
#Carregar e Preparar o Dataset

# Carregar o dataset (substitua pelo caminho correto do dataset MNIST)
train_data = h2o.import_file("mnist_train.csv")
test_data = h2o.import_file("mnist_test.csv")

# Especificar a coluna de resposta e as variáveis preditoras
response = "label"
predictors = train_data.columns[1:]  # Exclui a primeira coluna que é o rótulo


In [ ]:
#Configurar o Pré-Processamento dos Dados

# Transformar a coluna de rótulos em fator para classificação
train_data[response] = train_data[response].asfactor()
test_data[response] = test_data[response].asfactor()


In [ ]:
# Definir o Modelo de Deep Learning com H2O e Configurar a Grid Search

# Definir hiperparâmetros para a Grid Search
hyper_params = {
    "hidden": [[32, 32], [64, 64], [128, 128]],  # Diferentes combinações de camadas e neurónios
    "rate": [0.01, 0.005, 0.001],                # Diferentes taxas de aprendizagem
    "epochs": [10, 20, 30]                       # Diferentes números de épocas
}

# Criar o modelo de Deep Learning com H2O
dl_model = H2ODeepLearningEstimator(
    activation="RectifierWithDropout",  # Função de ativação com dropout
    input_dropout_ratio=0.2,            # Dropout para a camada de entrada
    hidden_dropout_ratios=[0.5, 0.5],   # Dropout nas camadas ocultas
    loss="CrossEntropy",                # Função de perda para classificação multiclasse
    distribution="multinomial"          # Distribuição para problemas de classificação
)

# Configurar a Grid Search
grid = H2OGridSearch(model=dl_model, hyper_params=hyper_params, grid_id="dl_grid")


In [ ]:
#Treinar o Modelo com Grid Search

# Executar a Grid Search
grid.train(x=predictors, y=response, training_frame=train_data, validation_frame=test_data)


In [ ]:
#Analisar os Resultados da Grid Search

# Exibir os resultados da Grid Search e ordenar por acurácia de validação
grid_results = grid.get_grid(sort_by="accuracy", decreasing=True)
print(grid_results)



In [ ]:
#Avaliar o Melhor Modelo

# Selecionar o melhor modelo
best_model = grid_results.models[0]

# Avaliar o melhor modelo no conjunto de teste
performance = best_model.model_performance(test_data)
print(performance)


In [ ]:
# Encerrar a sessão do H2O
h2o.shutdown()

##Resumo
1.  Instalar e inicializar o H2O.
2.  Carregar e preparar os dados: Carregue os dados no formato H2O e transforme a coluna de rótulo em categórica.
3.  Configurar os hiperparâmetros para a Grid Search: Defina as combinações de camadas, taxas de aprendizagem e épocas.
4.  Executar a Grid Search para encontrar a melhor configuração.
5.  Avaliar o melhor modelo selecionado com o conjunto de teste.